In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import random
from sklearn.metrics import r2_score

# LECTURA Y TRANSFORMACIÓN DATASET

#Carga datos

#Lectura y transformación datos dataset de diabetes
diabetes = pd.read_csv('datasets/diabetes_normalized.dat')
Diabetes = pd.DataFrame(diabetes).to_numpy()

#Lectura y transformación datos dataset de laser
laser = pd.read_csv('datasets/laser_normalized.dat')
Laser = pd.DataFrame(laser).to_numpy()

#Lectura y transformacioón datos dataset de terremotos
quake = pd.read_csv('datasets/quake_normalized.dat')
Quake = pd.DataFrame(quake).to_numpy()

In [2]:
#Función para obetener las variables de entrenamiento del dataset
def obtenerDatosEntrenamiento(datos):
    datos_sin_eliminados = datos[:,1:-1]
    return datos_sin_eliminados

In [3]:
#Función para obetener la variable objetivo del dataset
def obtenerDatosSalida(datos):
    datos_sin_eliminados = datos[:,-1].flatten()
    return datos_sin_eliminados

In [4]:
#Diferneciar array

#Variables de entrenamiento Diabetes
datosEntrenamientoDiabetes = obtenerDatosEntrenamiento(Diabetes)

#Variable objetivo Diabetes
datosSalidaDiabetes = obtenerDatosSalida(Diabetes)


#Variables de entrenamiento Laser
datosEntrenamientoLaser = obtenerDatosEntrenamiento(Laser)

#Variable objetivo Laser
datosSalidaLaser = obtenerDatosSalida(Laser)


#Variables de entrenamiento terremotos
datosEntrenamientoQuake = obtenerDatosEntrenamiento(Quake)

#Variable objetivo terremotos
datosSalidaQuake = obtenerDatosSalida(Quake)

## POBLACIÓN

In [5]:
#Funcion para generar población para cualquier tamaño y dato de entrada
def CrearPoblacion(X,numIndividuos):
    #poblacion = np.array([[random.uniform(-1, 1) for _ in range(X.shape[1]+1)] for _ in range(numIndividuos)])
    poblacion = np.random.uniform(-1, 1, size=(numIndividuos, X.shape[1]+1))
    return poblacion

## FUNCIONES FITNESS

In [6]:
#Función fitness de error cuadrático medio
def Fitness_ECM(poblacion, datosEntrenamiento, datosSalida):
    
    #Donde guardar la poblacion fit
    poblacion_fit = {} 
        
    #Para cada cromosoma de la poblacion
    for j,cromosoma in enumerate(poblacion):
        diferencia = 0
        
        #Calcular la distancia a cada una de las variables de entrenamientos
        for i,x in enumerate(datosEntrenamiento):
            
            #Distancia
            predicciones = (x[0]*cromosoma[2]) + (x[1]*cromosoma[1]) + cromosoma[0]
            
            #Calcular la diferencia sumando las medias de las diferencias
            diferencia = diferencia + np.mean((predicciones - datosSalida[i])**2)
        
        #Guardar las inversas de las diferencias con respecto a la poblacion
        poblacion_fit[j] = 1/(diferencia/len(datosEntrenamiento))
        
    return poblacion_fit

In [7]:
#Funcion fitness de coeficiente de determinación
def FitnessR2(poblacion, datosEntrenamiento, datosSalida):
    
    #Donde guardar la poblacion fit
    fitness = {}
    
    #Para cada individuo de la poblacion
    for i,individuo in enumerate(poblacion):
        
        #Realizzar la prediccion
        prediccion = np.dot(datosEntrenamiento, individuo[1:]) + individuo[0]
        
        #Obtener fitness
        r2Score = r2_score(datosSalida, prediccion)
        
        #Si es mayor que 0, se multiplica por 10
        if r2Score > 0:
            fitness[i] = round(r2Score, 5) * 10
            
        #Si es menor que 0, se obtiene el absoluto de la inversa
        else:
            fitness[i] = abs(round(1 / r2Score, 5))
            
    return fitness

## SELECCIÓN DE PROGENITORES

In [8]:
#Selección de progenitores con el método de la ruleta
def SelProgenitoresRuleta(poblacion,datosEntrenamiento, datosSalida):
    
    #Realizar el fitness de toda la poblacion
    diferencias = Fitness_ECM(poblacion,datosEntrenamiento, datosSalida)

    ruleta = []
    
    #Normalizar el fitness
    ruleta.append([value/np.sum(list(diferencias.values())) for key,value in diferencias.items()])
    ruleta = ruleta[0]
    
    #Donde se van a guardar los intervalos
    intervalos = []
    
    #Introducir el primer intervalo
    intervalos.append(np.round(ruleta[0],5))
    
    #Introducir el resto de intervalos
    for i in range(1,len(ruleta)):
        intervalos.append(np.round(intervalos[i-1] + ruleta[i],5))
    
    #Donde se van a guardar los progenitores
    progenitores = []
    
    #Por cada elemento de la población, un progenitor
    for i in range(len(poblacion)):
        
        #Número aleatorio entre 0 y 1
        aleatorio = np.random.rand()
        
        #Para cada intervalo
        for i, intervalo in enumerate(intervalos):
            
            #Si estoy en el intervalo introducir el ejemplo
            if aleatorio < intervalo:
                progenitores.append(i)
                break
                
    return [poblacion[i] for i in progenitores]

In [9]:
#Selección de progenitores con el método del torneo
def SelProgenitoresTorneo(poblacion, datosEntrenamiento, datosSalida):
    
    #Donde se van a guardar los progenitores
    progenitores = []
    
    #Fitness de la población
    fitness = Fitness_ECM(poblacion, datosEntrenamiento, datosSalida)
    
    #Por cada elemento de la población, un progenitor
    for _ in range(len(poblacion)):  
        
        #Los rivales obtinen valores aleatorios entre o y el tamaño de la poblacion menos uno
        rival1 = random.randint(0, len(poblacion) - 1)
        rival2 = random.randint(0, len(poblacion) - 1)
        
        #Los rivales no pueden ser iguales
        while rival2 == rival1:
            rival2 = random.randint(0, len(poblacion) - 1)
        
        #Si el fitness del primer valor es mayor que el fitness del segundo rival, 
        #selecconarlo como progenitor, en caso contrario escoger al segundo
        if fitness[rival1] >= fitness[rival2]:
            progenitores.append(poblacion[rival1])
        else:
            progenitores.append(poblacion[rival2])

    return progenitores

In [10]:
#Selección de progenitores con el método del ranking
def SelProgenitoresRanking(poblacion, datosEntrenamiento, datosSalida):
    
    #Fitness de la poblacion
    fitness = Fitness_ECM(poblacion, datosEntrenamiento, datosSalida)
    
    num_padres = len(poblacion)
    
    orden = np.argsort(fitness)  # Obtener el orden ascendente de las aptitudes
    ranking = np.arange(len(poblacion), 0, -1)  # Crear un arreglo con los rankings
    probabilidades = ranking / np.sum(ranking)  # Calcular las probabilidades de selección
    padres_indices = np.random.choice(len(poblacion), size=num_padres, replace=False, p=probabilidades)
    padres = [poblacion[i] for i in padres_indices]
    return padres

## CRUZAMIENTO

In [11]:
#Cruzamiento por un punto
def CruzamientoUnPunto(cromosoma1, cromosoma2, prob_cruz):
    
    #Si un numero aleatorio es menor que la probabilidad de cruce
    aleatorio = np.random.rand()
    if aleatorio < prob_cruz:
        x = cromosoma1.shape
        
        #Una dimension
        cromosoma1 = cromosoma1.flatten()
        cromosoma2 = cromosoma2.flatten()
        
        #Punto de corte al azar
        crossover_point = np.random.randint(1, len(cromosoma1))
        
        #Obtener los dos hijos
        child1 = np.concatenate((cromosoma1[0:crossover_point], cromosoma2[crossover_point:])).reshape(x)
        child2 = np.concatenate((cromosoma2[0:crossover_point], cromosoma1[crossover_point:])).reshape(x)
        return child1, child2
    
    else:
        return cromosoma1, cromosoma2

In [12]:
#Cruzamiento aritmetico
def CruzamientoAritmetico(cromosoma1, cromosoma2, prob_cruz, alpha):
    
    #Si un numero aleatorio es menor que la probabilidad de cruce
    aleatorio = np.random.rand()
    if aleatorio < prob_cruz:
        
        #Una dimension
        cromosoma1 = cromosoma1.flatten()
        cromosoma2 = cromosoma2.flatten()
        
        aux = np.copy(cromosoma1)
        cromosoma1 = alpha*cromosoma1 + (1-alpha) * cromosoma2
        cromosoma2 = alpha*cromosoma2 + (1-alpha) * aux
        return cromosoma1, cromosoma2
    
    else:
        return cromosoma1, cromosoma2

## MUTACIÓN

In [13]:
#Método de mutación
def Mutacion(cromosoma, prob_mut):
    
    #Para cada cromosoma
    for i in range(len(cromosoma)):
        aleatorio = np.random.rand()
        
        #Si el numero aleatrorio es mentor que la probabilidad de mutación, introducir un cromosoma aleatorio
        if(aleatorio<prob_mut):
            aleatorio2 = np.random.rand()
            cromosoma[i] = aleatorio2
    return cromosoma

## SELECCIÓN DE SUPERVIVIENTES

In [14]:
#Selección de supervivientes
def SelSupervivientes(poblacion, nuevos, datosEntrenamiento, datosSalida):
    supervivientes = []
    
    #Cada elemento de la población añadirlo a supervivientes
    for i in range(len(poblacion)):
        supervivientes.append(poblacion[i])
    
    #Cada elemento de nuevos coromosomas añadirlo a supervivientes
    for i in range(len(nuevos)):
        supervivientes.append(nuevos[i])
        
    numero_supervivientes = len(supervivientes)//2
    
    #Fitness para la nueva mejor poblacion
    mejor_poblacion = Fitness_ECM(supervivientes, datosEntrenamiento, datosSalida)
    
    #Ordenar mejor poblacion
    poblacion_ordenada = sorted(mejor_poblacion.items(), key =lambda x:x[1], reverse = True)
    
    #Ordenar los indices en función de la mejor poblacion ordenada
    indices_ordenados = [elemento[0] for elemento in poblacion_ordenada]
    
    #Insertar en supervivientes los mejores cormosomas
    resultado = []
    for i in range(numero_supervivientes):
        resultado.append(supervivientes[indices_ordenados[i]])
        
    return resultado

## ALGORITMO GENERAL

In [15]:
def main(datosEntrenamiento, datosSalida, prob_cruz = 0.95, prob_mut = 0.01, num_iteraciones = 2000, tamañoPoblacion = 10):
    
    #Generar población aleatoria
    poblacion = CrearPoblacion(datosEntrenamiento,tamañoPoblacion)
    
    #Definir supervivientes
    supervivientes = poblacion
    
    #Bucle del algoritmo
    for iter in range(num_iteraciones):
        
        #Definir poblacion
        poblacion = supervivientes

        #Seleccion de progenitores
        progenitores = SelProgenitoresRuleta(poblacion,datosEntrenamiento, datosSalida)
        
        #Cruzamiento
        nuevos = []
        for i in range(0,len(progenitores),2):
            nuevo1,nuevo2=CruzamientoUnPunto(progenitores[i],progenitores[i+1],prob_cruz)
            nuevos.append(nuevo1)
            nuevos.append(nuevo2)

        #Mutacion
        for i in range(len(nuevos)):
            nuevos[i] = Mutacion(nuevos[i], prob_mut)

        #Seleccion de supervivientes
        supervivientes = SelSupervivientes(poblacion,nuevos, datosEntrenamiento, datosSalida)



    fitness_final = Fitness_ECM(supervivientes, datosEntrenamiento, datosSalida)
    resultado = np.array(supervivientes[np.argmax(fitness_final)])
    return resultado

In [16]:
def mostrarGrafica(X,y,mejor_cromosoma,X_label = '',Y_label = '',Z_label = ''):
    
    #Generar 10 puntos para la primera dimensión (B0) entre 0 y 1
    puntos = np.linspace(0,1,10)
    
    #Generar 7 puntos para la segunda dimensión (B1) entre 0 y 1
    puntos2 = np.linspace(0,1,7)

    # Crear todas las combinaciones de los puntos creados
    xx, yy = np.meshgrid(puntos,puntos2)

    # Crea una matriz Xtest que contenga todos los valores de xx en la primera columna y los de yy en la segunda
    Xtest = np.hstack((xx.reshape(-1,1),yy.reshape(-1,1)))

    # Utiliza predict para obtener los valores de Z (el plano) para despues dibujarlo
    regr = LinearRegression()
    regr.fit(X,y)
    Z = regr.predict(Xtest)
    Z = Z.reshape(xx.shape)

    #Calcular la predicción con los pesos del cromosoma obtenido por el algoritmo genético
    regr.coef_ = mejor_cromosoma[1:]
    Z2 = regr.predict(Xtest)
    Z2 = Z2.reshape(xx.shape)

    #Pintar las dos gráficas
    fig = plt.figure(figsize=(15,6))
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122, projection='3d')

    #Pintar los puntos y el plano
    ax1.scatter(X[:,0], X[:,1], y.ravel(),c='r')
    ax1.plot_surface(xx,yy,Z,alpha=0.3)
    ax1.set_title('Regresión lineal')

    # Nombrar los ejes
    ax1.set_xlabel(X_label)
    ax1.set_ylabel(Y_label)
    ax1.set_zlabel(Z_label)

    #Pintar los puntos y el plano
    ax2.scatter(X[:,0], X[:,1], y.ravel(),c='r')
    ax2.plot_surface(xx,yy,Z2,alpha=0.3)
    ax2.set_title('Algoritmo genético')

    #Nombrar los ejes
    ax2.set_xlabel(X_label)
    ax2.set_ylabel(Y_label)
    ax2.set_zlabel(Z_label)

In [ ]:
X_diabetes = obtenerDatosEntrenamiento(Diabetes)
y_diabetes = obtenerDatosSalida(Diabetes)

mejorCromosoma = np.array(main(datosEntrenamientoDiabetes, datosSalidaDiabetes))
mostrarGrafica(X_diabetes, y_diabetes, mejorCromosoma)

In [ ]:
print(mejorCromosoma)